In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)


In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Creatomh our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# Display the row's' columns and data in dictionary format for
# Measurement table
first_row = session.query(Measurement).first()
first_row.__dict__

In [ ]:
# Display the row's column and data in dictionary format for 
# Station table
first_row = session.query(Station).first()
first_row.__dict__

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database.

# Calculate the date one year from the last date in data set.
  #since latest date is 2017-08-23, date one year from last date should be 
    # 2016-08-23

# Perform a query to retrieve the data and precipitation scores
prcp_1yr = session.query(Measurement.date, Measurement.prcp).\
             filter(Measurement.date >'2016-08-23').\
                order_by(Measurement.date).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
prcp_df = pd.DataFrame(prcp_1yr, columns = ['date', 'prcp'])
prcp_df=prcp_df.fillna(0)
# prcp_df = prcp_df.groupby("date").sum()
prcp_df=prcp_df.set_index("date" )



# Sort the dataframe by date

prcp_df = prcp_df.sort_index()


# Use Pandas Plotting with Matplotlib to plot the data
prcp_df.plot.bar(width = 20)
# plt.tight_layout()
plt.locator_params(nbins = 12)

plt.ylabel("Precipitation")
plt.xlabel("Date")
plt.savefig("Images/Precipitation.png")

plt.show()


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcp_df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
no_of_station = session.query(Station.station).count()
print(f" The total number of statioin is {no_of_station}")


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
most_active = session.query(Measurement.station, func.count(Measurement.station)).\
              group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
# .group_by(Station.station).count()
most_active

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
lowest_temp = session.query(Measurement.tobs).filter(Measurement.station == 'USC00519281').\
                order_by(Measurement.tobs).first()
print(f"lowest temperature at the station was {lowest_temp}")

highest_temp = session.query(Measurement.tobs).filter(Measurement.station == 'USC00519281').\
                order_by(Measurement.tobs.desc()).first()
print(f"Highest temperature at the station was {highest_temp}")

avg_temp = session.query(func.avg(Measurement.tobs)).filter(Measurement.station == 'USC00519281').all()
print(f"Average temperature at the station was {avg_temp}")

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
active_1yr = session.query(Measurement.date, Measurement.tobs).\
             filter(Measurement.date >'2016-08-23').\
                order_by(Measurement.date).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
active_df = pd.DataFrame(active_1yr, columns = ['date', 'Temperature'])

active_df=active_df.set_index("date" )
active_df.head()

In [ ]:
#Plotting the temperature data for most active station using histogram
ax = active_df.plot.hist(bins=12, alpha=0.8)
plt.xlabel("Temperature")
plt.ylabel("frequency")
plt.savefig("Images/Temperature.png")

# Close session

In [ ]:

 #test cell 

In [ ]:
# Close Session
session.close()